In [1]:
import pandas as pd
from tqdm import tqdm

### EDA

In [11]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope and credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/home/ameyh/cred/google.json', scope)
client = gspread.authorize(credentials)

# Open the Google Sheet by its title
sheet = client.open('Counter Speech Project - Dashboard')

In [17]:
subsheet1 = sheet.worksheet('tagged-dataset (IntentConan original)')
df_original_dataset = pd.DataFrame(subsheet1.get_all_records())
print(df_original_dataset.shape)
df_original_dataset.head(2)

(14124, 10)


,hatespeech,csType,counterspeech,Suggest,Contexual Relevance,Aggressive,Complexity,Comments,source,Paper
0,Maybe the UN could talk to those asian and afr...,Informative,The US is the second most polluting country in...,3,4,2,3,,Human,
1,Maybe the UN could talk to those asian and afr...,Questioning,Doesn't everyone on the planet have a responsi...,1,3,2,2,,Human,


In [18]:
# subsheet2 = sheet.worksheet('untagged-dataset')
# df_new_dataset = pd.DataFrame(subsheet2.get_all_records())
# df_new_dataset.head(2)

In [31]:
df_original_dataset['source'].value_counts()

Human    13945
Name: source, dtype: int64

In [40]:
df_original_dataset.drop_duplicates()
df_original_dataset = df_original_dataset.drop_duplicates(subset=['hatespeech', 'csType'], keep='last').reset_index(drop=True)
df_original_dataset.shape


(13945, 10)

In [41]:
df_original_dataset['source'] = 'Human'

In [42]:
df_pred_dialoGPT = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/baseline-systems/baseline-DialoGPT.csv')
print(df_pred_dialoGPT.shape)
df_pred_dialoGPT = df_pred_dialoGPT.drop(columns=['counterspeech'])
df_pred_dialoGPT = df_pred_dialoGPT.rename(columns={
    'predicted_counterspeech': 'counterspeech'
})
df_pred_dialoGPT['source'] = 'dialoGPT'
display(df_pred_dialoGPT)

(2971, 7)


,id,Unnamed: 0,hate_speech_with_csType,counterspeech,hatespeech,csType,source
0,0,0,"Women are basically childlike, they remain thi...",There are many cases of women being raped by m...,"Women are basically childlike, they remain thi...",Informative,dialoGPT
1,1,1,"Women are basically childlike, they remain thi...",This statement is completely false and offensi...,"Women are basically childlike, they remain thi...",Denouncing,dialoGPT
2,2,2,"Women are basically childlike, they remain thi...",I understand that you may have some concerns a...,"Women are basically childlike, they remain thi...",Positive,dialoGPT
3,3,3,Minorities vote to strip wealth and power from...,People's right to vote is not determined by ra...,Minorities vote to strip wealth and power from...,Informative,dialoGPT
4,4,4,Minorities vote to strip wealth and power from...,Can you provide evidence to support this claim...,Minorities vote to strip wealth and power from...,Questioning,dialoGPT
...,...,...,...,...,...,...,...
2966,2966,2966,The Holocaust never happened. It was a coverup...,The holocaust happened. it's a sad chapter in ...,The Holocaust never happened. It was a coverup...,Informative,dialoGPT
2967,2967,2967,We cannot coexist with millions of Muslims in ...,I understand that you may have concerns about ...,We cannot coexist with millions of Muslims in ...,Positive,dialoGPT
2968,2968,2968,"Women are basically childlike, they remain thi...",The idea that women are childlike is a stereot...,"Women are basically childlike, they remain thi...",Informative,dialoGPT
2969,2969,2969,"Women do not have equal rights, they are subhu...",What evidence supports the claim that women ar...,"Women do not have equal rights, they are subhu...",Questioning,dialoGPT


In [43]:
df_pred_quarc = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/baseline-systems/baseline-QUARC.csv')
print(df_pred_quarc.shape)
df_pred_quarc = df_pred_quarc.drop(columns=['counterspeech'])
df_pred_quarc = df_pred_quarc.rename(columns={
    'predicted_counterspeech': 'counterspeech'
})
df_pred_quarc['source'] = 'QUARC'
display(df_pred_quarc)

(2971, 26)


,level_0,index,hatespeech,csType,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,hatespeechImplication,targetGroupEmotionalReaction,targetGroupCognitiveReaction,hatespeechOffensiveness,id,is_high_quality,hs_id,hatespeechTarget,powerDynamics,counterspeech
0,0,20,"Women are basically childlike, they remain thi...",Informative,3.0,4.0,2.0,3.0,NaN,QUARC,...,"implies that all women are childlike, soft, an...",women could feel angry that their gender is be...,"will feel less welcome in the conversation, wi...",offensive generalization,20,no,5,women,gender,This is not true. women are just as capable as...
1,1,22,"Women are basically childlike, they remain thi...",Denouncing,2.0,3.0,3.0,2.0,NaN,QUARC,...,"implies that all women are childlike, soft, an...",women could feel angry that their gender is be...,"will feel less welcome in the conversation, wi...",offensive generalization,22,no,5,women,gender,This statement perpetuates harmful stereotypes...
2,2,23,"Women are basically childlike, they remain thi...",Positive,2.0,2.0,2.0,2.0,NaN,QUARC,...,"implies that all women are childlike, soft, an...",women could feel angry that their gender is be...,"will feel less welcome in the conversation, wi...",offensive generalization,23,no,5,women,gender,If you have any questions or want to discuss t...
3,3,48,Minorities vote to strip wealth and power from...,Informative,3.0,4.0,3.0,3.0,NaN,QUARC,...,implies that minorities are organized to take ...,"minorities could feel angry, frustrated, or sc...",might lead minorities to be more cautious when...,offensive,52,no,12,jews,religion,"Minorities are a part of our society, and they..."
4,4,49,Minorities vote to strip wealth and power from...,Questioning,1.0,1.0,3.0,1.0,NaN,QUARC,...,implies that minorities are organized to take ...,"minorities could feel angry, frustrated, or sc...",might lead minorities to be more cautious when...,offensive,53,no,12,jews,religion,Is it fair to generalize an entire racial grou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2966,2966,13925,The Holocaust never happened. It was a coverup...,Informative,NaN,NaN,NaN,NaN,NaN,QUARC,...,implies that Jews are responsible for the Holo...,Jews could feel angry at being blamed for some...,"Jews could likely be more feared, and will fee...",offensive generalization,12328,no,3483,jews,religion,The holocaust was a tragic event that resulted...
2967,2967,13933,We cannot coexist with millions of Muslims in ...,Positive,3.0,3.0,3.0,2.0,NaN,QUARC,...,implies that Muslims are not compatible with t...,"Muslims could feel angry, scared, and unwelcom...","Muslims could feel less welcome in the West, m...",offensive generalization,3555,no,1908,muslims,religion,"All individuals, regardless of their religious..."
2968,2968,13942,"Women are basically childlike, they remain thi...",Informative,3.0,4.0,3.0,2.0,NaN,QUARC,...,implies that women are not capable of thinking...,"women could feel angry, frustrated, and disres...",might lead women to feel less confident in the...,offensive generalization,3458,no,1847,women,gender,This is not true. women are just as capable as...
2969,2969,13944,"Women do not have equal rights, they are subhu...",Questioning,NaN,NaN,NaN,NaN,NaN,QUARC,...,implies that women are not worthy of equal rig...,"women could feel angry, frustrated, and disres...","will feel less welcome in society, will want t...",offensive generalization,13787,no,3486,women,gender,Why do you believe that all women are subhuman...


In [44]:
df_pred_gps = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/baseline-systems/baseline-GPS.csv')
print(df_pred_gps.shape)
df_pred_gps = df_pred_gps.drop(columns=['counterspeech'])
df_pred_gps = df_pred_gps.rename(columns={
    'predicted_counterspeech': 'counterspeech'
})
df_pred_gps['source'] = 'GPS'
display(df_pred_gps)

(2971, 8)


,Unnamed: 0,hate_speech,counterspeech,csType,hatespeech,id,hs_id,source
0,0,"Women are basically childlike, they remain thi...","women are not childlike, they have necessarily...",Informative,"Women are basically childlike, they remain thi...",20,5,GPS
1,1,"Women are basically childlike, they remain thi...","women are not childlike, they have necessarily...",Denouncing,"Women are basically childlike, they remain thi...",22,5,GPS
2,2,"Women are basically childlike, they remain thi...","women are not childlike, they have necessarily...",Positive,"Women are basically childlike, they remain thi...",23,5,GPS
3,3,Minorities vote to strip wealth and power from...,jews are not very founded for a thing? they ar...,Informative,Minorities vote to strip wealth and power from...,52,12,GPS
4,4,Minorities vote to strip wealth and power from...,jews are not very founded for a thing? they ar...,Questioning,Minorities vote to strip wealth and power from...,53,12,GPS
...,...,...,...,...,...,...,...,...
2966,2966,The Holocaust never happened. It was a coverup...,"The holocaust was the systematic, bureaucratic...",Informative,The Holocaust never happened. It was a coverup...,12328,3483,GPS
2967,2967,We cannot coexist with millions of Muslims in ...,There is no evidence that muslims are incompat...,Positive,We cannot coexist with millions of Muslims in ...,3555,1908,GPS
2968,2968,"Women are basically childlike, they remain thi...","women are not childlike, they have necessarily...",Informative,"Women are basically childlike, they remain thi...",3458,1847,GPS
2969,2969,"Women do not have equal rights, they are subhu...",women are subhuman and everyone deserves equal...,Questioning,"Women do not have equal rights, they are subhu...",13787,3486,GPS


In [45]:
keep_cols = ['hatespeech', 'counterspeech', 'csType', 'source']
df_combined = pd.concat([df_original_dataset[keep_cols], df_pred_dialoGPT[keep_cols], df_pred_gps[keep_cols], df_pred_quarc[keep_cols]], axis=0)
print(df_combined.shape)
display(df_combined)

(22858, 4)


,hatespeech,counterspeech,csType,source
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human
1,Maybe the UN could talk to those asian and afr...,Doesn't everyone on the planet have a responsi...,Questioning,Human
2,Maybe the UN could talk to those asian and afr...,The world would be a better place if people we...,Denouncing,Human
3,Maybe the UN could talk to those asian and afr...,"You're right, ocean pollution is one of the ma...",Positive,Human
4,‘Juice (Jews) are worse that nukes.’ Caption t...,Anti-semitism is a serious problem that we nee...,Informative,Human
...,...,...,...,...
2966,The Holocaust never happened. It was a coverup...,The holocaust was a tragic event that resulted...,Informative,QUARC
2967,We cannot coexist with millions of Muslims in ...,"All individuals, regardless of their religious...",Positive,QUARC
2968,"Women are basically childlike, they remain thi...",This is not true. women are just as capable as...,Informative,QUARC
2969,"Women do not have equal rights, they are subhu...",Why do you believe that all women are subhuman...,Questioning,QUARC


In [46]:
df_combined['source'].value_counts()

Human       13945
dialoGPT     2971
GPS          2971
QUARC        2971
Name: source, dtype: int64

In [48]:
df_combined = df_combined.drop_duplicates(subset=['counterspeech'], keep='last').reset_index(drop=True)
print(df_combined.shape)
df_combined['source'].value_counts()

(19556, 4)


Human       13824
dialoGPT     2914
QUARC        1900
GPS           918
Name: source, dtype: int64

In [49]:
df_combined.to_csv('/home/ameyh/counterspeech-EVAL/data/data_pass2.csv', index=False)

In [51]:
df_combined[df_combined.source != 'Human'].to_csv('/home/ameyh/counterspeech-EVAL/data/data_pass2_nonHuman.csv', index=False)

In [9]:
# Further Analysis
df = pd.read_csv('/home/ameyh/counterspeech-EVAL/data/results_combined_suitableness_nonHuman.csv')
print(df.shape)
df.head(1)

(5732, 27)


,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),...,"aq_score_(hs,cs)","bert_score_(hs,cs)",toxicity_score_(cs),obscene_score(cs),identity_attack_score(cs),insult_score(cs),"bm25_(hs,cs)","mauve_(hs,cs)",prompt_suitableness_score,prediction_(prompt_suitableness_score)_(gpt-3.5-turbo)
0,"Women are basically childlike, they remain thi...",There are many cases of women being raped by m...,Informative,dialoGPT,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,NaN,{ \n'Coherence': '3'\n},...,0.813657,0.861275,0.287367,0.005473,0.049298,0.008135,-0.545166,namespace(divergence_curve=array([[1.00000000e...,\nYou will be given one counterspeech (also ca...,{ \n'Choose or not': '2'\n}


In [6]:
len(df.hatespeech.unique())

1095

In [12]:
df['(csType,source)'] = df.apply(lambda row: f"({row['csType']}, {row['source']})", axis=1)
df['(csType,source)'].value_counts()

(csType,source)
(Positive, dialoGPT)       741
(Denouncing, dialoGPT)     737
(Informative, dialoGPT)    735
(Questioning, dialoGPT)    701
(Informative, QUARC)       648
(Positive, QUARC)          644
(Denouncing, QUARC)        454
(Questioning, GPS)         299
(Positive, GPS)            288
(Denouncing, GPS)          197
(Questioning, QUARC)       154
(Informative, GPS)         134
Name: count, dtype: int64

In [19]:
df_ = df.groupby(['hatespeech', 'csType'])
list_df = df_.agg(list)
list_df

counterspeech  \
hatespeech                                         csType                                                           
Above all Islam is a religion of conquest.         Denouncing   [It is unfair and inaccurate to label an entir...   
                                                   Informative  [In fact, islam is a diverse religion with ove...   
                                                   Positive     [I understand that you may have concerns about...   
According to the Bible, then gays are an abomin... Denouncing   [This statement is a harmful stereotype. it is...   
                                                   Positive     [I understand that you may have strong beliefs...   
...                                                                                                           ...   
the Nazis were right Jews were responsible for ... Denouncing   [This statement is offensive and perpetuates h...   
                                                   Positive     [I understand that you may have strong opinion...   
                                                   Questioning  [Do you have any evidence for this?, The idea ...   
women are basically childlike, incapable of ser... Positive     [I understand that you may have some misconcep...   
                                                   Questioning  [Shouldn't women be given equal opportunities ...   

                                                                                source  \
hatespeech                                         csType                                
Above all Islam is a religion of conquest.         Denouncing        [dialoGPT, QUARC]   
                                                   Informative         [dialoGPT, GPS]   
                                                   Positive          [dialoGPT, QUARC]   
According to the Bible, then gays are an abomin... Denouncing               [dialoGPT]   
                                                   Positive          [dialoGPT, QUARC]   
...                                                                                ...   
the Nazis were right Jews were responsible for ... Denouncing        [dialoGPT, QUARC]   
                                                   Positive     [dialoGPT, GPS, QUARC]   
                                                   Questioning  [dialoGPT, GPS, QUARC]   
women are basically childlike, incapable of ser... Positive          [dialoGPT, QUARC]   
                                                   Questioning              [dialoGPT]   

                                                                                           prompt_relevance_score  \
hatespeech                                         csType                                                           
Above all Islam is a religion of conquest.         Denouncing   [\nYou will be given one counterspeech (also c...   
                                                   Informative  [\nYou will be given one counterspeech (also c...   
                                                   Positive     [\nYou will be given one counterspeech (also c...   
According to the Bible, then gays are an abomin... Denouncing   [\nYou will be given one counterspeech (also c...   
                                                   Positive     [\nYou will be given one counterspeech (also c...   
...                                                                                                           ...   
the Nazis were right Jews were responsible for ... Denouncing   [\nYou will be given one counterspeech (also c...   
                                                   Positive     [\nYou will be given one counterspeech (also c...   
                                                   Questioning  [\nYou will be given one counterspeech (also c...   
women are basically childlike, incapable of ser... Positive     [\nYou will be given one counterspeech (also c...   
   

### Aggregate multiple runs

In [3]:
df1 = pd.read_csv('/home/ameyh/counterspeech-EVAL/data/results_data_pass2_aggressiveness.csv')
df1.head(1)

,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4)
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n}


In [4]:
df2 = pd.read_csv('/home/ameyh/counterspeech-EVAL/data/results_data_pass2_coherence.csv')
df2.head(1)

,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_coherence_score)_(gpt-3.5-turbo)
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Coherence': '4'\n}


In [5]:
df3_1 = pd.read_csv('/home/ameyh/counterspeech-EVAL/data/results_data_pass2_nonHuman_relevance.csv')
print(df3_1.shape)
df3_1.head(1)

(5732, 7)


,hatespeech,counterspeech,csType,source,prompt_relevance_score,system_description,prediction_gpt-4
0,"Women are basically childlike, they remain thi...",There are many cases of women being raped by m...,Informative,dialoGPT,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Relevance': '1'\n}


In [6]:
df3_2 = pd.read_csv('/home/ameyh/counterspeech-EVAL/data/results_data_pass1_relevance.csv')
df3_2 = df3_2[[col for col in df3_2.columns if col in df3_1.columns]]
print(df3_2.shape)
df3_2.head(1)

(5998, 7)


,hatespeech,csType,counterspeech,source,prompt_relevance_score,system_description,prediction_gpt-4
0,Maybe the UN could talk to those asian and afr...,Informative,The us is the second most polluting country in...,Human,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Relevance': '5'\n}


In [7]:
df3 = pd.concat([df3_1, df3_2])
df3.head(1)

,hatespeech,counterspeech,csType,source,prompt_relevance_score,system_description,prediction_gpt-4
0,"Women are basically childlike, they remain thi...",There are many cases of women being raped by m...,Informative,dialoGPT,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Relevance': '1'\n}


In [8]:
df_merged = pd.merge(df1, df2)
print(df_merged.shape)
df_merged.head(1)

(19556, 10)


,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo)
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n}


In [9]:
df_merged = pd.merge(df_merged, df3[['hatespeech', 'counterspeech','prompt_relevance_score','prediction_gpt-4']], on=['hatespeech', 'counterspeech', 'prompt_relevance_score'], how='left')
print(df_merged.shape)
df_merged.head(1)

(19556, 11)


,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),prediction_gpt-4
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},NaN


In [10]:
df_merged = df_merged.rename(columns={
    'prediction_gpt-4': 'prediction_(prompt_relevance_score)_(gpt-4)'
})
# df_merged = df_merged.drop(columns=['prediction_gpt-4'])

df_merged.to_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv', index=False)

### Compute Metrics

In [2]:
from metrics import Metrics
metrics = Metrics()

/home/ameyh/.conda/envs/cobra-frames/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/ameyh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using device: cuda


Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
input_df = pd.read_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv')
input_df.head(1)

,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),...,"pc_score_(hs,cs)","cd_score_(hs,cs)","aq_score_(hs,cs)","bert_score_(hs,cs)",toxicity_score_(cs),obscene_score(cs),identity_attack_score(cs),insult_score(cs),"bm25_(hs,cs)","mauve_(hs,cs)"
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},...,0.990696,0.328718,0.598695,0.862366,0.000784,0.000033,0.000128,0.000171,-0.885247,namespace(divergence_curve=array([[1.00000000e...


In [4]:
input_df['mauve_(hs,cs)'].iloc[2]

'namespace(divergence_curve=array([[1.00000000e+00, 0.00000000e+00],\n       [9.99995726e-01, 1.73609007e-26],\n       [8.32453186e-01, 1.04224880e-06],\n       [6.86004679e-01, 2.12658472e-05],\n       [5.59564799e-01, 1.25514223e-04],\n       [4.51537272e-01, 4.44552693e-04],\n       [3.60174000e-01, 1.18857004e-03],\n       [2.83705856e-01, 2.65777018e-03],\n       [2.20406964e-01, 5.25063195e-03],\n       [1.68630394e-01, 9.47002044e-03],\n       [1.26829057e-01, 1.59271474e-02],\n       [9.35679711e-02, 2.53432440e-02],\n       [6.75310811e-02, 3.85486758e-02],\n       [4.75243925e-02, 5.64790718e-02],\n       [3.24764845e-02, 8.01678174e-02],\n       [2.14370692e-02, 1.10733938e-01],\n       [1.35740346e-02, 1.49363885e-01],\n       [8.16926692e-03, 1.97284880e-01],\n       [4.61345404e-03, 2.55725967e-01],\n       [2.40000897e-03, 3.25860097e-01],\n       [1.11820438e-03, 4.08714815e-01],\n       [4.45573425e-04, 5.05026324e-01],\n       [1.39597172e-04, 6.14979187e-01],\n      

In [5]:
input_df = input_df[input_df.counterspeech.notna()]

In [6]:
assert input_df['hatespeech'].notnull().all()
assert input_df['counterspeech'].notnull().all()

In [7]:
hatespeech = input_df.hatespeech.values.tolist()
counterspeech = input_df.counterspeech.values.tolist()

In [8]:
mauve_scores = metrics.compute_mauve(
    predictions=counterspeech, references=hatespeech
)
input_df['mauve_(hs,cs)'] = mauve_scores

ValueError: Mismatch in the number of predictions (244) and references (171)

In [17]:
print(mauve_scores.mauve)


0.01043245041350205


In [16]:
for x in mauve_scores:
    print(x)

TypeError: 'types.SimpleNamespace' object is not iterable

In [8]:
bm25_scores = metrics.compute_bm25(
    predictions=counterspeech, references=hatespeech
)
input_df['bm25_(hs,cs)'] = bm25_scores

In [10]:
cosine_similarity_scores = metrics.compute_cosine_similarity(
    predictions=counterspeech, references=hatespeech
)
input_df['cosine_similarity_(hs,cs)'] = cosine_similarity_scores

KeyboardInterrupt: 

In [19]:
bert_scores = metrics.compute_bertscore(
    predictions=counterspeech, references=hatespeech
)
input_df['bert_score_(hs,cs)'] = bert_scores

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
(
    toxicity_score,
    obscene_score,
    identity_attack_score,
    insult_score,
) = metrics.compute_toxicity_score(predictions=counterspeech)

input_df['toxicity_score_(cs)'] = toxicity_score
input_df['obscene_score(cs)'] = obscene_score
input_df['identity_attack_score(cs)'] = identity_attack_score
input_df['insult_score(cs)'] = insult_score

Running Toxicity inference on 19555 data points: 100%|██████████| 1223/1223 [01:00<00:00, 20.27it/s]


In [8]:
input_df.to_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv', index=False)

In [13]:
pc_score, cd_score, aq_score = metrics.compute_pc_cd_aq_scores(
    hatespeech=hatespeech, counterspeech=counterspeech, batch_size=250
)

input_df['pc_score_(hs,cs)'] = pc_score
input_df['cd_score_(hs,cs)'] = cd_score
input_df['aq_score_(hs,cs)'] = aq_score

Running CA inference on 19555 data points:   0%|          | 0/79 [00:00<?, ?it/s]





ProConClient: 100%|██████████| 250/250 [00:01<00:00, 175.02it/s]






ClaimDetectionClient: 100%|██████████| 250/250 [00:01<00:00, 205.79it/s]






Running CA inference on 19555 data points:   1%|▏         | 1/79 [00:03<05:02,  3.88s/it]





ProConClient: 100%|██████████| 250/250 [00:03<00:00, 68.62it/s]






ClaimDetectionClient: 100%|██████████| 250/250 [00:01<00:00, 204.60it/s]






Running CA inference on 19555 data points:   3%|▎         | 2/79 [00:10<06:45,  5.27s/it]





ProConClient: 100%|██████████| 250/250 [00:03<00:00, 74.63it/s]



Running CA inference on 19555 data points:   3%|▎         | 2/79 [00:54<35:12, 27.44s/it]


KeyboardInterrupt: 

### Analysis of Metrics

In [34]:
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import MinMaxScaler

def calculate_correlations_with_scaling(df, col1, col2):
    """
    Calculate Spearman's rho and Kendall's tau for two columns in a DataFrame after scaling.

    Parameters:
    - df: pandas.DataFrame containing the data.
    - col1: str, name of the first column.
    - col2: str, name of the second column.

    Returns:
    - spearman_rho: Spearman's rho coefficient after scaling.
    - kendall_tau: Kendall's tau coefficient after scaling.
    """
    # Initialize the MinMaxScaler to scale between -1 and 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    
    # Scale the specified columns
    scaled_values = scaler.fit_transform(df[[col1, col2]])
    
    # Update the DataFrame with the scaled values
    df_scaled = pd.DataFrame(scaled_values, columns=[col1, col2])
    
    # Calculate Spearman's rho on the scaled data
    spearman_rho, spearman_p_value = spearmanr(df_scaled[col1], df_scaled[col2])
    
    # Calculate Kendall's tau on the scaled data
    kendall_tau, kendall_p_value = kendalltau(df_scaled[col1], df_scaled[col2])
    
    return spearman_rho, kendall_tau


def calculate_correlation_matrices(df, automated_metrics, human_ratings):
    """
    Calculate Spearman's rho and Kendall's tau matrices for multiple automated metrics against human ratings.

    Parameters:
    - df: pandas.DataFrame containing the data.
    - automated_metrics: list of str, names of the automated metric columns.
    - human_ratings: list of str, names of the human rating columns.

    Returns:
    - spearman_df: DataFrame containing Spearman's rho values.
    - kendall_df: DataFrame containing Kendall's tau values.
    """
    spearman_results = []
    kendall_results = []
    
    # Iterate over each combination of automated metric and human rating
    for metric in automated_metrics:
        spearman_row = []
        kendall_row = []
        for rating in human_ratings:
            # spearman_rho, _ = spearmanr(df[metric], df[rating])
            # kendall_tau, _ = kendalltau(df[metric], df[rating])
            spearman_rho, kendall_tau = calculate_correlations_with_scaling(df, metric, rating)
            spearman_row.append(spearman_rho)
            kendall_row.append(kendall_tau)
        
        spearman_results.append(spearman_row)
        kendall_results.append(kendall_row)
        
    # Convert the results to DataFrames
    spearman_df = pd.DataFrame(spearman_results, index=automated_metrics, columns=human_ratings)
    kendall_df = pd.DataFrame(kendall_results, index=automated_metrics, columns=human_ratings)
    
    return spearman_df, kendall_df


df = pd.read_csv('/home/ameyh/counterspeech-EVAL/preds/results_combined.csv')
print(df.shape)
print(df.columns)
df.head(1)

(19555, 26)
Index(['hatespeech', 'counterspeech', 'csType', 'source',
       'prompt_relevance_score', 'prompt_aggressiveness_score',
       'prompt_coherence_score', 'system_description',
       'prediction_(prompt_aggressiveness_score)_(gpt-4)',
       'prediction_(prompt_coherence_score)_(gpt-3.5-turbo)',
       'prediction_(prompt_relevance_score)_(gpt-4)', 'relevance_score',
       'aggressiveness_score', 'coherence_score', 'cosine_similarity_(hs,cs)',
       'pc_score_(hs,cs)', 'cd_score_(hs,cs)', 'aq_score_(hs,cs)',
       'bert_score_(hs,cs)', 'toxicity_score_(cs)', 'obscene_score(cs)',
       'identity_attack_score(cs)', 'insult_score(cs)', 'bm25_(hs,cs)',
       'mauve_(hs,cs)', 'prompt_suitableness_score'],
      dtype='object')


,hatespeech,counterspeech,csType,source,prompt_relevance_score,prompt_aggressiveness_score,prompt_coherence_score,system_description,prediction_(prompt_aggressiveness_score)_(gpt-4),prediction_(prompt_coherence_score)_(gpt-3.5-turbo),...,"cd_score_(hs,cs)","aq_score_(hs,cs)","bert_score_(hs,cs)",toxicity_score_(cs),obscene_score(cs),identity_attack_score(cs),insult_score(cs),"bm25_(hs,cs)","mauve_(hs,cs)",prompt_suitableness_score
0,Maybe the UN could talk to those asian and afr...,The US is the second most polluting country in...,Informative,Human,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,\nYou will be given one counterspeech (also ca...,NaN,{ \n'Aggressiveness': '1'\n},{ \n'Coherence': '4'\n},...,0.328718,0.598695,0.862366,0.000784,0.000033,0.000128,0.000171,-0.885247,namespace(divergence_curve=array([[1.00000000e...,\nYou will be given one counterspeech (also ca...


In [35]:
automated_metrics = ['cosine_similarity_(hs,cs)', 'pc_score_(hs,cs)', 'cd_score_(hs,cs)', 'aq_score_(hs,cs)', 'bert_score_(hs,cs)', 'toxicity_score_(cs)', 'obscene_score(cs)', 'identity_attack_score(cs)', 'insult_score(cs)', 'bm25_(hs,cs)']
human_ratings = ['aggressiveness_score', 'coherence_score', 'relevance_score']
spearman_df, kendall_df = calculate_correlation_matrices(df, automated_metrics, human_ratings)

In [36]:
spearman_df

,aggressiveness_score,coherence_score,relevance_score
"cosine_similarity_(hs,cs)",-0.022471,-0.021855,0.135229
"pc_score_(hs,cs)",0.019044,-0.002858,-0.122101
"cd_score_(hs,cs)",-0.030412,0.092180,0.214933
"aq_score_(hs,cs)",-0.011959,0.063111,0.117461
"bert_score_(hs,cs)",-0.003598,-0.033322,0.118342
toxicity_score_(cs),0.047184,-0.198503,0.082120
obscene_score(cs),0.041185,-0.161205,0.036505
identity_attack_score(cs),0.016699,-0.148496,0.121109
insult_score(cs),0.055329,-0.199020,0.073687
"bm25_(hs,cs)",0.038542,-0.095435,-0.105799


In [37]:
kendall_df

,aggressiveness_score,coherence_score,relevance_score
"cosine_similarity_(hs,cs)",-0.018024,-0.016572,0.108529
"pc_score_(hs,cs)",0.015242,-0.002104,-0.097772
"cd_score_(hs,cs)",-0.024388,0.073251,0.171635
"aq_score_(hs,cs)",-0.009539,0.050146,0.094402
"bert_score_(hs,cs)",-0.002871,-0.026568,0.094572
toxicity_score_(cs),0.037778,-0.157951,0.065459
obscene_score(cs),0.032918,-0.127994,0.029210
identity_attack_score(cs),0.013395,-0.117997,0.096500
insult_score(cs),0.044299,-0.158360,0.058662
"bm25_(hs,cs)",0.030869,-0.075890,-0.084834


In [20]:
import pandas as pd
from scipy.stats import spearmanr, kendalltau

def calculate_correlation_matrices(df, automated_metrics, human_ratings):
    """
    Calculate Spearman's rho and Kendall's tau matrices for multiple automated metrics against human ratings.

    Parameters:
    - df: pandas.DataFrame containing the data.
    - automated_metrics: list of str, names of the automated metric columns.
    - human_ratings: list of str, names of the human rating columns.

    Returns:
    - spearman_df: DataFrame containing Spearman's rho values.
    - kendall_df: DataFrame containing Kendall's tau values.
    """
    spearman_results = []
    kendall_results = []
    
    # Iterate over each combination of automated metric and human rating
    for metric in automated_metrics:
        spearman_row = []
        kendall_row = []
        for rating in human_ratings:
            spearman_rho, _ = spearmanr(df[metric], df[rating])
            kendall_tau, _ = kendalltau(df[metric], df[rating])
            
            spearman_row.append(spearman_rho)
            kendall_row.append(kendall_tau)
        
        spearman_results.append(spearman_row)
        kendall_results.append(kendall_row)
    
    # Convert the results to DataFrames
    spearman_df = pd.DataFrame(spearman_results, index=automated_metrics, columns=human_ratings)
    kendall_df = pd.DataFrame(kendall_results, index=automated_metrics, columns=human_ratings)
    
    return spearman_df, kendall_df

# Example usage
if __name__ == "__main__":
    # Create a sample DataFrame
    data = {
        'automated_metric_1': [0.1, 0.4, 0.35, 0.8, 0.9],
        'automated_metric_2': [0.2, 0.5, 0.55, 0.85, 0.95],
        'automated_metric_3': [0.15, 0.45, 0.4, 0.78, 0.88],
        'human_rating_1': [1, 2, 3, 4, 5],
        'human_rating_2': [2, 3, 2, 5, 4]
    }
    df = pd.DataFrame(data)
    
    # Specify the columns for automated metrics and human ratings
    automated_metrics = ['automated_metric_1', 'automated_metric_2', 'automated_metric_3']
    human_ratings = ['human_rating_1', 'human_rating_2']
    
    # Calculate correlation matrices
    spearman_df, kendall_df = calculate_correlation_matrices(df, automated_metrics, human_ratings)
    
    print("Spearman's rho matrix:")
    print(spearman_df)
    print("\nKendall's tau matrix:")
    print(kendall_df)


Spearman's rho matrix:
                    human_rating_1  human_rating_2
automated_metric_1             0.9        0.872082
automated_metric_2             1.0        0.718185
automated_metric_3             0.9        0.872082

Kendall's tau matrix:
                    human_rating_1  human_rating_2
automated_metric_1             0.8        0.737865
automated_metric_2             1.0        0.527046
automated_metric_3             0.8        0.737865


In [1]:
from bleurt import score

def calculate_bleurt_score(reference, candidate):
    scorer = score.BleurtScorer("bleurt-base-128")
    scores = scorer.score([reference], [candidate])
    return scores["scores"][0]

ModuleNotFoundError: No module named 'bleurt'

### CTC - Score

In [1]:
import nltk
nltk.download('stopwords')

from ctc_score import StyleTransferScorer, SummarizationScorer, DialogScorer

# SummEval
doc = "(CNN)Donald Sterling's racist remarks cost him an NBA team last year. But now it's his former female companion who has lost big. A Los Angeles judge has ordered V. Stiviano to pay back more than $2.6 million in gifts after Sterling's wife sued her. In the lawsuit, Rochelle \"Shelly\" Sterling accused Stiviano of targeting extremely wealthy older men. She claimed Donald Sterling used the couple's money to buy Stiviano a Ferrari, two Bentleys and a Range Rover, and that he helped her get a $1.8 million duplex. Who is V. Stiviano? Stiviano countered that there was nothing wrong with Donald Sterling giving her gifts and that she never took advantage of the former Los Angeles Clippers owner, who made much of his fortune in real estate. Shelly Sterling was thrilled with the court decision Tuesday, her lawyer told CNN affiliate KABC. \"This is a victory for the Sterling family in recovering the $2,630,000 that Donald lavished on a conniving mistress,\" attorney Pierce O'Donnell said in a statement. \"It also sets a precedent that the injured spouse can recover damages from the recipient of these ill-begotten gifts.\" Stiviano's gifts from Donald Sterling didn't just include uber-expensive items like luxury cars. According to the Los Angeles Times, the list also includes a $391 Easter bunny costume, a $299 two-speed blender and a $12 lace thong. Donald Sterling's downfall came after an audio recording surfaced of the octogenarian arguing with Stiviano. In the tape, Sterling chastises Stiviano for posting pictures on social media of her posing with African-Americans, including basketball legend Magic Johnson. \"In your lousy f**ing Instagrams, you don't have to have yourself with -- walking with black people,\" Sterling said in the audio first posted by TMZ. He also tells Stiviano not to bring Johnson to Clippers games and not to post photos with the Hall of Famer so Sterling's friends can see. \"Admire him, bring him here, feed him, f**k him, but don't put (Magic) on an Instagram for the world to have to see so they have to call me,\" Sterling said. NBA Commissioner Adam Silver banned Sterling from the league, fined him $2.5 million and pushed through a charge to terminate all of his ownership rights in the franchise. Fact check: Donald Sterling's claims vs. reality CNN's Dottie Evans contributed to this report."
hypo = "-lrb- cnn -rrb- donald sterling 's racist remarks cost him an nba team last year . but now it 's his former female companion who has lost big . a los angeles judge has ordered v. stiviano to pay back more than $ 2.6 million in gifts after sterling 's wife sued her ."

scorer = SummarizationScorer(align='D-cnndm')

score = scorer.score(doc=doc, refs=[], hypo=hypo, aspect='consistency')
print(score)

[nltk_data] Downloading package stopwords to /home/ameyh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/ameyh/.conda/envs/cs-eval/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: No token found in warning page from Google Drive.

### UniEval

In [1]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'summarization'

# a list of source documents
src_list = ['Peter and Elizabeth took a taxi to attend the night party in the city. \
             While in the party, Elizabeth collapsed and was rushed to the hospital.']
# a list of human-annotated reference summaries
ref_list = ['Elizabeth was hospitalized after attending a party with Peter.']
# a list of model outputs to be evaluataed
output_list = ['Peter and Elizabeth attend party city. Elizabeth rushed hospital.']

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, 
                       src_list=src_list, ref_list=ref_list)
# Initialize evaluator for a specific task
evaluator = get_evaluator(task)
# Get multi-dimensional evaluation scores
eval_scores = evaluator.evaluate(data, print_result=True)

/home/ameyh/.conda/envs/cs-eval/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'convert_to_json' from 'utils' (/home/ameyh/counterspeech-EVAL/utils.py)

### Take a deep breath, and redo the dataset

In [2]:
df_test = pd.read_csv('/home/ameyh/depository/Counter-Speech-Generation/data/m2/test.csv')
df_test.head(1)

,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,relevantPowerDynamics,hatespeechImplication,targetGroupEmotionalReaction,targetGroupCognitiveReaction,hatespeechOffensiveness,id,is_high_quality,hs_id,hatespeechTarget,powerDynamics
0,20,"Women are basically childlike, they remain thi...",Informative,The characteristics you are referring to are a...,3.0,4.0,2.0,3.0,NaN,Human,...,gender power differential between speaker and ...,"implies that all women are childlike, soft, an...",women could feel angry that their gender is be...,"will feel less welcome in the conversation, wi...",offensive generalization,20,no,5,women,gender


In [3]:
df_pred_dialoGPT = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/baseline-systems/baseline-DialoGPT.csv')
print(df_pred_dialoGPT.shape)
# df_pred_dialoGPT = df_pred_dialoGPT.drop(columns=['counterspeech'])
# df_pred_dialoGPT = df_pred_dialoGPT.rename(columns={
#     'predicted_counterspeech': 'counterspeech'
# })
df_pred_dialoGPT['source'] = 'dialoGPT'
df_pred_dialoGPT.head(1)

(2971, 7)


,id,Unnamed: 0,hate_speech_with_csType,predicted_counterspeech,counterspeech,hatespeech,csType,source
0,0,0,"Women are basically childlike, they remain thi...",There are many cases of women being raped by m...,The characteristics you are referring to are a...,"Women are basically childlike, they remain thi...",Informative,dialoGPT


In [4]:
df_pred_quarc = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/baseline-systems/baseline-QUARC.csv')
print(df_pred_quarc.shape)
# df_pred_quarc = df_pred_quarc.drop(columns=['counterspeech'])
# df_pred_quarc = df_pred_quarc.rename(columns={
#     'predicted_counterspeech': 'counterspeech'
# })
df_pred_quarc['source'] = 'QUARC'
df_pred_quarc.head(1)

(2971, 26)


,level_0,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,...,hatespeechImplication,targetGroupEmotionalReaction,targetGroupCognitiveReaction,hatespeechOffensiveness,id,is_high_quality,hs_id,hatespeechTarget,powerDynamics,predicted_counterspeech
0,0,20,"Women are basically childlike, they remain thi...",Informative,The characteristics you are referring to are a...,3.0,4.0,2.0,3.0,NaN,...,"implies that all women are childlike, soft, an...",women could feel angry that their gender is be...,"will feel less welcome in the conversation, wi...",offensive generalization,20,no,5,women,gender,This is not true. women are just as capable as...


In [5]:
df_pred_gps = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/baseline-systems/baseline-GPS.csv')
print(df_pred_gps.shape)
# df_pred_gps = df_pred_gps.drop(columns=['counterspeech'])
# df_pred_gps = df_pred_gps.rename(columns={
#     'predicted_counterspeech': 'counterspeech'
# })
df_pred_gps['source'] = 'GPS'
df_pred_gps.head(1)

(2971, 8)


,Unnamed: 0,hate_speech,predicted_counterspeech,csType,hatespeech,counterspeech,id,hs_id,source
0,0,"Women are basically childlike, they remain thi...","women are not childlike, they have necessarily...",Informative,"Women are basically childlike, they remain thi...",The characteristics you are referring to are a...,20,5,GPS


In [6]:
df_pred_gpt_turbo_zs = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/prompting/prompting-GPT-3.5-ZS.csv')
print(df_pred_gpt_turbo_zs.shape)
# df_pred_gpt_turbo_zs = df_pred_gpt_turbo_zs.drop(columns=['counterspeech'])
# df_pred_gpt_turbo_zs = df_pred_gpt_turbo_zs.rename(columns={
#     'predicted_counterspeech': 'counterspeech'
# })
df_pred_gpt_turbo_zs['source'] = 'GPT3.5_ZS'
df_pred_gpt_turbo_zs.head(1)

(2971, 28)


,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,hatespeechOffensiveness,id,is_high_quality,hs_id,hatespeechTarget,powerDynamics,prompt,system_description,gpt_prediction,predicted_counterspeech
0,20,"Women are basically childlike, they remain thi...",Informative,The characteristics you are referring to are a...,3.0,4.0,2.0,3.0,NaN,GPT3.5_ZS,...,offensive generalization,20,no,5,women,gender,»»»» Definition »»»»\ncounterspeech: countersp...,NaN,The statement you made about women being child...,The statement you made about women being child...


In [7]:
df_pred_gpt_turbo_fs = pd.read_csv('/home/ameyh/counterspeech-ARR-2023/predictions/prompting/prompting-GPT-3.5-FS.csv')
print(df_pred_gpt_turbo_fs.shape)
# df_pred_gpt_turbo_fs = df_pred_gpt_turbo_fs.drop(columns=['counterspeech'])
# df_pred_gpt_turbo_fs = df_pred_gpt_turbo_fs.rename(columns={
#     'predicted_counterspeech': 'counterspeech'
# })
df_pred_gpt_turbo_fs['source'] = 'GPT3.5_FS'
df_pred_gpt_turbo_fs.head(1)

(2971, 29)


,index,hatespeech,csType,counterspeech,Suggest,Relevance,Aggressive,Complexity,Comments,source,...,id,is_high_quality,hs_id,hatespeechTarget,powerDynamics,prompt,exemplars,system_description,gpt_prediction,predicted_counterspeech
0,20,"Women are basically childlike, they remain thi...",Informative,The characteristics you are referring to are a...,3.0,4.0,2.0,3.0,NaN,GPT3.5_FS,...,20,no,5,women,gender,»»»» Definition »»»»\ncounterspeech: countersp...,"statement: ""Women are basically childlike, the...",NaN,"""Actually, the idea that women are childlike a...","Actually, the idea that women are childlike an..."


In [8]:
keep_cols = ['hatespeech', 'counterspeech', 'predicted_counterspeech', 'csType', 'source']
df_combined = pd.concat([df_pred_dialoGPT[keep_cols], df_pred_gps[keep_cols], df_pred_quarc[keep_cols], df_pred_gpt_turbo_zs[keep_cols], df_pred_gpt_turbo_fs[keep_cols]], axis=0)
print(df_combined.shape)
display(df_combined)

(14855, 5)


,hatespeech,counterspeech,predicted_counterspeech,csType,source
0,"Women are basically childlike, they remain thi...",The characteristics you are referring to are a...,There are many cases of women being raped by m...,Informative,dialoGPT
1,"Women are basically childlike, they remain thi...",Nobody apart from people who spread misinforma...,This statement is completely false and offensi...,Denouncing,dialoGPT
2,"Women are basically childlike, they remain thi...",It is a question of survival and happiness to ...,I understand that you may have some concerns a...,Positive,dialoGPT
3,Minorities vote to strip wealth and power from...,This is incorrect. jews could never be able to...,People's right to vote is not determined by ra...,Informative,dialoGPT
4,Minorities vote to strip wealth and power from...,Where is your evidence?,Can you provide evidence to support this claim...,Questioning,dialoGPT
...,...,...,...,...,...
2966,The Holocaust never happened. It was a coverup...,The holocaust was a genocide committed by the ...,The claim that the Holocaust never happened is...,Informative,GPT3.5_FS
2967,We cannot coexist with millions of Muslims in ...,It's important to remember that diversity and ...,I understand that you may have concerns about ...,Positive,GPT3.5_FS
2968,"Women are basically childlike, they remain thi...",This claim is not supported by any evidence or...,NaN,Informative,GPT3.5_FS
2969,"Women do not have equal rights, they are subhu...",Isn't it important to promote equality and res...,Can you provide evidence or examples to suppor...,Questioning,GPT3.5_FS


In [9]:
df_combined.describe()

,hatespeech,counterspeech,predicted_counterspeech,csType,source
count,14855,14855,14692,14855,14855
unique,1097,2969,11508,4,5
top,Children should learn about gender identity an...,"If we didn't have open borders, then there wou...",Is it fair to generalize an entire group of pe...,Informative,dialoGPT
freq,20,10,131,3735,2971


In [10]:
# Identify the duplicate rows in the 'col1' column
mask = df_combined.duplicated(subset='predicted_counterspeech')
# Replace the duplicate values in the 'col1' column with NaN
df_combined.loc[mask, 'predicted_counterspeech'] = -1

df_combined


,hatespeech,counterspeech,predicted_counterspeech,csType,source
0,"Women are basically childlike, they remain thi...",The characteristics you are referring to are a...,There are many cases of women being raped by m...,Informative,dialoGPT
1,"Women are basically childlike, they remain thi...",Nobody apart from people who spread misinforma...,This statement is completely false and offensi...,Denouncing,dialoGPT
2,"Women are basically childlike, they remain thi...",It is a question of survival and happiness to ...,I understand that you may have some concerns a...,Positive,dialoGPT
3,Minorities vote to strip wealth and power from...,This is incorrect. jews could never be able to...,People's right to vote is not determined by ra...,Informative,dialoGPT
4,Minorities vote to strip wealth and power from...,Where is your evidence?,Can you provide evidence to support this claim...,Questioning,dialoGPT
...,...,...,...,...,...
2966,The Holocaust never happened. It was a coverup...,The holocaust was a genocide committed by the ...,The claim that the Holocaust never happened is...,Informative,GPT3.5_FS
2967,We cannot coexist with millions of Muslims in ...,It's important to remember that diversity and ...,I understand that you may have concerns about ...,Positive,GPT3.5_FS
2968,"Women are basically childlike, they remain thi...",This claim is not supported by any evidence or...,-1,Informative,GPT3.5_FS
2969,"Women do not have equal rights, they are subhu...",Isn't it important to promote equality and res...,Can you provide evidence or examples to suppor...,Questioning,GPT3.5_FS


In [11]:
df_view = df_combined[df_combined['predicted_counterspeech'] != -1]
df_view.source.value_counts()

# (We need to fill all -1 data points with predictions from QUARC)

source
dialoGPT     2918
GPT3.5_FS    2892
GPT3.5_ZS    2885
QUARC        1896
GPS           918
Name: count, dtype: int64

In [19]:
df_view.source.value_counts()

source
dialoGPT     2918
GPT3.5_FS    2892
GPT3.5_ZS    2885
QUARC        1896
GPS           918
Name: count, dtype: int64

In [20]:
df_grouped = df_view.groupby(['hatespeech', 'csType', 'counterspeech']).aggregate(list)
df_grouped

predicted_counterspeech  \
hatespeech                                         csType      counterspeech                                                                                           
Above all Islam is a religion of conquest.         Denouncing  No, islam is a religion of 1.6 billion people f...  [It is unfair and inaccurate to label an entir...   
                                                   Informative The definition of 'religion of conquest' is a p...  [In fact, islam is a diverse religion with ove...   
                                                   Positive    Islam is a religion of peaceful coexistence, th...  [I understand that you may have concerns about...   
                                                   Questioning Do you have any evidence for saying this?           [How can you say that about an entire religion...   
According to the Bible, then gays are an abomin... Denouncing  Have you even read the bible? there is no evide...  [This statement is a harmful stereotype. it is...   
...                                                                                                                                                              ...   
the Nazis were right Jews were responsible for ... Denouncing  This statement is a baseless and offensive cons...  [This statement is offensive and perpetuates h...   
                                                   Positive    I understand that you have strong beliefs, but ...  [I understand that you may have strong opinion...   
                                                   Questioning Why do you believe that same-sex attraction is ...  [Do you have any evidence for this?, 1. Can yo...   
women are basically childlike, incapable of ser... Positive    Any difference between men and women is a part ...  [I understand that you may have some misconcep...   
                                                   Questioning Don't you think that your argument is flawed?       [Shouldn't women be given equal opportunities ...   

                                                                                                                                                         source  
hatespeech                                         csType      counterspeech                                                                                     
Above all Islam is a religion of conquest.         Denouncing  No, islam is a religion of 1.6 billion people f...  [dialoGPT, GPS, QUARC, GPT3.5_ZS, GPT3.5_FS]  
                                                   Informative The definition of 'religion of conquest' is a p...  [dialoGPT, GPS, QUARC, GPT3.5_ZS, GPT3.5_FS]  
                                                   Positive    Islam is a religion of peaceful coexistence, th...  [dialoGPT, GPS, QUARC, GPT3.5_ZS, GPT3.5_FS]  
                                                   Questioning Do you have any evidence for saying this?                [dialoGPT, QUARC, GPT3.5_ZS, GPT3.5_FS]  
According to the Bible, then gays are an abomin... Denouncing  Have you even read the bible? there is no evide...              [dialoGPT, GPT3.5_ZS, GPT3.5_FS]  
...                                                                                                                                                         ...  
the Nazis were right Jews were responsible for ... Denouncing  This statement is a baseless and offensive cons...  [dialoGPT, GPS, QUARC, GPT3.5_ZS, GPT3.5_FS]  
                                                   Positive    I understand that you have strong beliefs, but ...       [dialoGPT, QUARC, GPT3.5_ZS, GPT3.5_FS]  
                                                   Questioning Why do you believe that same-sex attraction is ...              [dialoGPT, GPT3.5_ZS, GPT3.5_FS]  
women are basically childlike, incapable of ser... Positive    Any difference between men and women is a part ...                  [dialoGPT, QUARC, GPT3.5_ZS]  
            

In [25]:
df_grouped['pred_len'] = df_grouped['predicted_counterspeech'].apply(lambda x: len(x))
df_grouped['pred_len'].value_counts()

pred_len
4    1438
3     843
5     617
2      70
1       3
Name: count, dtype: int64

In [27]:
df_grouped[df_grouped['pred_len'] <= 2]

predicted_counterspeech  \
hatespeech                                         csType      counterspeech                                                                                           
All Muslims are incompatible with any peaceful ... Questioning Are you suggesting that every single muslim is ...  [can we the positive judgments about migrants....   
All Muslims are terrorists, they are all incomp... Questioning Have you interacted with every single muslim? a...  [Can you provide evidence or examples to suppo...   
Children with Down Syndrome are a burden to soc... Positive    People with down syndrome should be taught in m...  [I understand that you may have concerns about...   
Faggots are not welcome among us humans because... Positive    Embracing diversity and inclusion is what makes...  [I understand that you may have some concerns ...   
                                                   Questioning Why should someone's difference in sexual orien...  [Is it fair to generalize and label all femini...   
...                                                                                                                                                              ...   
We must stop Islam and replace it with a new re... Questioning Why do you think it is necessary to stop islam ...  [Why should someone's worth be determined by t...   
Women are a burden to society and should not be... Questioning How can we ensure equal representation and deci...  [Why do you believe that women are not capable...   
Women are basically childlike, incapable of cri... Questioning Have you considered the countless accomplished ...  [Is it fair to generalize the actions and inte...   
Women are basically childlike, they remain this... Informative This claim is not supported by any evidence or ...  [The idea that women are childlike is a stereo...   
Women are subhuman and are created to please men.  Questioning How can anyone justify the dehumanization of a ...  [Can you provide any evidence or logical reaso...   

                                                                                                                                   source  \
hatespeech                                         csType      counterspeech                                                                
All Muslims are incompatible with any peaceful ... Questioning Are you suggesting that every single muslim is ...        [GPS, GPT3.5_ZS]   
All Muslims are terrorists, they are all incomp... Questioning Have you interacted with every single muslim? a...  [GPT3.5_ZS, GPT3.5_FS]   
Children with Down Syndrome are a burden to soc... Positive    People with down syndrome should be taught in m...   [dialoGPT, GPT3.5_FS]   
Faggots are not welcome among us humans because... Positive    Embracing diversity and inclusion is what makes...   [dialoGPT, GPT3.5_ZS]   
                                                   Questioning Why should someone's difference in sexual orien...   [dialoGPT, GPT3.5_FS]   
...                                                                                                                                   ...   
We must stop Islam and replace it with a new re... Questioning Why do you think it is necessary to stop islam ...   [dialoGPT, GPT3.5_FS]   
Women are a burden to society and should not be... Questioning How can we ensure equal representation and deci...   [dialoGPT, GPT3.5_ZS]   
Women are basically childlike, incapable of cri... Questioning Have you considered the countless accomplished ...   [dialoGPT, GPT3.5_FS]   
Women are basically childlike, they remain this... Informative This claim is not supported by any evidence or ...   [dialoGPT, GPT3.5_ZS]   
Women are subhuman and are created to please men.  Questioning How can anyone justify the dehumanization of a ...  [GPT3.5_ZS, GPT3.5_FS]   

                                                                                                                   pred_le

In [14]:
df_combined.to_csv('/home/ameyh/counterspeech-EVAL/final_data/primary_dataset.csv', index=False)